<div style='color: lightblue;
           background-color: black;
           font-size: 200%;
           border-radius:15px;
           text-align:center;
           font-weight:600;
           border-style: dashed;
           border-color: lightblue;
           font-family: "Verdana";'>
Import Packages
<a class="anchor" id="1"></a> 

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import string
import re
from nltk.tokenize import word_tokenize

import gensim
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
import pyLDAvis
import pyLDAvis.gensim_models

<div style='color: lightblue;
           background-color: black;
           font-size: 200%;
           border-radius:15px;
           text-align:center;
           font-weight:600;
           border-style: dashed;
           border-color: lightblue;
           font-family: "Verdana";'>
Import Data
<a class="anchor" id="1"></a> 

In [2]:
df_ganjar = pd.read_csv(r"../Scraper\data_clean\userProfile_ganjarpranowo.csv")
df_prabowo = pd.read_csv(r"../Scraper\data_clean\userProfile_prabowo.csv")
df_anies = pd.read_csv(r"../Scraper\data_clean\userProfile_aniesbaswedan.csv")
df_ahy = pd.read_csv(r"../Scraper\data_clean\userProfile_AgusYudhoyono.csv")
df_ridwan = pd.read_csv(r"../Scraper\data_clean\userProfile_ridwankamil.csv")

<div style='color: lightblue;
           background-color: black;
           font-size: 200%;
           border-radius:15px;
           text-align:center;
           font-weight:600;
           border-style: dashed;
           border-color: lightblue;
           font-family: "Verdana";'>
LDA With SKlearn
<a class="anchor" id="1"></a> 

In [3]:
# Fungsi tokenize
df_ganjar['text_token'] = df_ganjar['clean_text_stem'].apply(lambda x: word_tokenize(str(x)))
df_prabowo['text_token'] = df_prabowo['clean_text_stem'].apply(lambda x: word_tokenize(str(x)))
df_anies['text_token'] = df_anies['clean_text_stem'].apply(lambda x: word_tokenize(str(x)))
df_ahy['text_token'] = df_ahy['clean_text_stem'].apply(lambda x: word_tokenize(str(x)))
df_ridwan['text_token'] = df_ridwan['clean_text_stem'].apply(lambda x: word_tokenize(str(x)))

In [4]:
def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, iterations=100)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        
    return model_list, coherence_values

In [5]:
dictionary = gensim.corpora.Dictionary(df_ganjar['text_token'])
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [6]:
bow_corpus = [dictionary.doc2bow(doc) for doc in df_ganjar['text_token']]

tfidf = gensim.models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [7]:
start=1
limit=11
step=1
model_list, coherence_values = compute_coherence_values(dictionary, corpus=corpus_tfidf, 
                                                        texts=df_ganjar['text_token'],
                                                        start=start, limit=limit, step=step)

x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 6))

In [ ]:
model = LdaModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=10) #num topic menyesuaikan hasil dari coherence value paling tinggi
for idx, topic in model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic)) 

In [ ]:
top_words_per_topic = []
for t in range(model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in model.show_topic(t, topn = 10)])
#pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("top_words.csv")
top_word = pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word','P'])
print(top_word)

In [ ]:
data = pyLDAvis.gensim_models.prepare(model, corpus_tfidf, dictionary)
pyLDAvis.save_html(data, 'lda-gensim-tfidf.html')